In [60]:
import aocd
import re
import numpy as np
import copy
%run helper.ipynb
puzzle = aocd.models.Puzzle(year=2024, day=15)
data = puzzle.input_data
import sys
sys.setrecursionlimit(6000)

In [27]:
data = """##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^"""

In [61]:
move_dict = {
    "<": Point2D((0, -1)),
    ">": Point2D((0, 1)),
    "^": Point2D((-1, 0)),
    "v": Point2D((1, 0)),
}

warehouse, moves = data.split("\n\n")
moves = [move_dict[m] for m in moves.replace("\n", "")]

In [56]:
def attempt_move(start, move, grid):
    curr = start
    while(True):
        next_loc = curr + move
        if grid[*next_loc] == "#":
            return start
        if grid[*next_loc] == ".":
            if curr == start:
                grid[*next_loc] = '@'
                grid[*curr] = "."
                return next_loc
            else:
                grid[*next_loc] = 'O'
                grid[*start] = '.'
                grid[*(start + move)] = '@'
                return start + move
        if grid[*next_loc] == "O":
            curr += move
            
def score(grid):
    boxes = np.transpose(np.where(grid == "O"))
    s = 0
    for box in boxes:
        s += 100 * box[0] + box[1]
    return s

In [57]:
grid = Grid(warehouse, str)
robot = Point2D(np.transpose(np.where(grid == '@'))[0])
for move in moves:
    robot = attempt_move(robot, move, grid)
puzzle.answer_a = int(score(grid))

In [62]:
def attempt_move_b(start, move, grid, curr):
#     print(start, move, grid[*curr], curr)
    if grid[*curr] == "#":
        return (False, [])
    if grid[*curr] == ".":
        return (True, [])
    if grid[*curr] == "@":
        possible, moves = attempt_move_b(start, move, grid, curr + move)
        if possible:
            return True, moves + [(curr + move, "@"), (curr, ".")]
        return False, []
    if grid[*curr] == "[" and move == Point2D((0, 1)): 
        possible, moves = attempt_move_b(start, move, grid, curr + move + move)
        if possible:
            return True, moves + [(curr + move + move, "]"), (curr + move, "[")]
        return False, []
    if grid[*curr] == "]" and move == Point2D((0, -1)): 
        possible, moves = attempt_move_b(start, move, grid, curr + move + move)
        if possible:
            return True, moves + [(curr + move + move, "["), (curr + move, "]")]
        return False, []
    if grid[*curr] == "[":
        other_side = curr + Point2D((0, 1))
    if grid[*curr] == "]":
        other_side = curr + Point2D((0, -1))
    possible_1, moves_1 = attempt_move_b(start, move, grid, curr + move)
    possible_2, moves_2 = attempt_move_b(start, move, grid, other_side + move)
    if possible_1 and possible_2:
        return (True, moves_1 + moves_2 + [
            (curr + move, grid[*(curr)]),
            (other_side + move, grid[*(other_side)]),
            (other_side, "."),
            (curr, ".")
        ])
    return False, []

def score_part_b(grid):
    boxes = np.transpose(np.where(grid == "["))
    s = 0
    for box in boxes:
        s += 100 * box[0] + box[1]
    return s

In [122]:
part_b_dict = {
    "#": "##",
    "O": "[]",
    ".": "..",
    "@": "@.",
    "\n": "\n"
}
part_b_warehouse = "".join([part_b_dict[c] for c in warehouse])
part_b_grid = Grid(part_b_warehouse, str)
part_b_robot = Point2D(np.transpose(np.where(part_b_grid == '@'))[0])
for move in moves[:1351]:
    print(move)
    possible, next_moves = attempt_move_b(part_b_robot, move, part_b_grid, part_b_robot)
    print(next_moves)
    if possible:
        part_b_robot += move
        for m in next_moves:
            part_b_grid[*m[0]] = m[1]
print(score_part_b(part_b_grid))

(1, 0)
[((25, 48), '@'), ((24, 48), '.')]
(1, 0)
[((26, 48), '@'), ((25, 48), '.')]
(1, 0)
[((28, 48), '['), ((28, 49), ']'), ((27, 49), '.'), ((27, 48), '.'), ((27, 48), '@'), ((26, 48), '.')]
(1, 0)
[((29, 48), '['), ((29, 49), ']'), ((28, 49), '.'), ((28, 48), '.'), ((28, 48), '@'), ((27, 48), '.')]
(0, -1)
[((28, 47), '@'), ((28, 48), '.')]
(0, 1)
[((28, 48), '@'), ((28, 47), '.')]
(0, -1)
[((28, 47), '@'), ((28, 48), '.')]
(1, 0)
[((29, 47), '@'), ((28, 47), '.')]
(0, -1)
[((29, 46), '@'), ((29, 47), '.')]
(0, 1)
[((29, 47), '@'), ((29, 46), '.')]
(1, 0)
[((30, 47), '@'), ((29, 47), '.')]
(1, 0)
[((33, 47), ']'), ((33, 46), '['), ((32, 46), '.'), ((32, 47), '.'), ((33, 46), '['), ((33, 47), ']'), ((32, 47), '.'), ((32, 46), '.'), ((32, 47), ']'), ((32, 46), '['), ((31, 46), '.'), ((31, 47), '.'), ((31, 47), '@'), ((30, 47), '.')]
(0, -1)
[((31, 46), '@'), ((31, 47), '.')]
(0, 1)
[((31, 47), '@'), ((31, 46), '.')]
(-1, 0)
[((30, 47), '@'), ((31, 47), '.')]
(1, 0)
[((31, 47), '@'), 

In [123]:
print('\n'.join([''.join(map(str,x)) for x in part_b_grid]))

####################################################################################################
##[][]##....[]##............##......[]..[]....####....[]..........[]......[]..............##......##
##......[]..[]......[]..[][]..[]........[]......[]..##......[]......[][][]##[]....##..##....[]..####
##....[]..[]..........[]..........[]....[][][]..##[][][]......[]........##..##..[][]........[][]..##
##..............[][]##........[][][][][]##......[]....##......##....[][]##[]..........[]..##......##
##............[]....[]........[]........[]............[]....[]........[]..[]##......##............##
##......[][][]....[]....[]....##....##..........##[]..[]....[]..........####....##....[]..[][]##..##
##....##[]..[]....[][][]....[][][]....[]..[]..[][]............[][][]..................[]....[]..[]##
##......[]..##......[]####[]........##[]....[][]..[]....[]..[]..[]........[]........##............##
##..........[]..[]....[][]..[]..........[]............[]..##[]......[][]..[]............[].

In [71]:
np.transpose(np.where(part_b_grid == '@'))[0]

array([23, 46])

In [121]:
print('\n'.join([''.join(map(str,x)) for x in part_b_grid]))

####################################################################################################
##[][]##....[]##............##......[]..[]....####....[]..........[]......[]..............##......##
##......[]..[]......[]..[][]..[]........[]......[]..##......[]......[][][]##[]....##..##....[]..####
##....[]..[]..........[]..........[]....[][][]..##[][][]......[]........##..##..[][]........[][]..##
##..............[][]##........[][][][][]##......[]....##......##....[][]##[]..........[]..##......##
##............[]....[]........[]........[]............[]....[]........[]..[]##......##............##
##......[][][]....[]....[]....##....##..........##[]..[]....[]..........####....##....[]..[][]##..##
##....##[]..[]....[][][]....[][][]....[]..[]..[][]............[][][]..................[]....[]..[]##
##......[]..##......[]####[]........##[]....[][]..[]....[]..[]..[]........[]........##............##
##..........[]..[]....[][]..[]..........[]............[]..##[]......[][]..[]............[].